In [1]:
## This will clean up the database and also generate MD5 sums for files in the UberDatabase
import os,sys,pymongo
import DSAHelperFunctions as dsa
from pprint import pprint
from bson.objectid import ObjectId
### Paths and Name for Archive and Location of Slides and Pyramids
archive_name= 'UberSliderSorter'  ### This obviously changes for a given slide set
## Create Database  Connections
client = pymongo.MongoClient('localhost',27017)
archive_db = client[archive_name]

## Within the ArrchiveDB I will have subcollections for SlideData, MetaData and Combined

In [ ]:
## http://twiecki.github.io/blog/2014/02/24/ipython-nb-cluster/ goof red
#http://blog.pythonisito.com/2012/01/moving-along-with-pymongo.html
# from IPython import parallel
# c = parallel.Client()
# view = c.load_balanced_view()

In [10]:
from IPython.parallel import Client
rc = Client()
dview = rc[:]
#https://ipython.org/ipython-doc/3/parallel/magics.html
@dview.parallel(block=False)
def md5sum(filename):
#     import hashlib
#     from functools import partial
    with open(filename, mode='rb') as f:
        d = hashlib.md5()
        for buf in iter(partial(f.read, 128), b''):
            d.update(buf)
    return d.hexdigest()

# some_complicated_function.map(arange(10))

In [6]:
%px import hashlib
%px from functools import partial
%px import pymongo

In [9]:
###http://stackoverflow.com/questions/11371009/parallel-mapping-functions-in-ipython-w-multiple-parameters
cl = 'RawSlideData.EMORY_BIOBANK'
slidesNeedMD5 = list(archive_db[cl].find({ 'md5Sum': {"$exists": False}}))
slidePathList = [x['slide_w_path'] for x in slidesNeedMD5]
print slidePathList[0:10]

[u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/11-1_PNC2.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/11-1_4NC.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/11-2_2NC.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/12-10_4PN.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/12-10_5NC.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/11-5_4NC.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/11-1_1NC.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/12-10_3PN.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/12-10_8margin.ndpi', u'/TCGA_MIRROR/EMORY_BIOBANK/Emory_BioBank/GBM_GLIOMA_PROJECT/12-10_margin.ndpi']


In [ ]:
##https://ipython.org/ipython-doc/3/parallel/magics.html
## Some magic operators to make this easier

In [14]:
ar = md5sum.map(slidePathList[0:100])  ## I've already connected the md5sum function with the dview /parallel code

In [15]:
ar.ready()

False

In [47]:
ar.ready()
with dview.sync_imports():
    import functools

importing functools on engine(s)


In [ ]:
dview.map_async(md5sum(),slidePathList[0:10])

In [ ]:
print dview

In [ ]:
##Other keys to flag--- noncofirming file name!!
### This will update/add md5sum for records if it doesn't have them
###Start computing the md5sum
for cl in archive_db.collection_names():
    md5_processed = 0
    if cl.startswith('RawSlideData'):
        print cl, archive_db[cl].count(),
        slidePtr = archive_db[cl].find({ 'md5Sum': {"$exists": False}})
        for r in slidePtr:
            #print r
            md5sum = dsa.md5sum(r['slide_w_path'])
            archive_db[cl].update_one({'_id': r['_id']}, { "$set": {'md5Sum': md5sum}})
            md5_processed +=1
            output = "Total md5 Processed: %d  " % (md5_processed)
            dsa.LinePrinter(output)
        print
        

RawSlideData.WINSHIP_BIOBANK 10
RawSlideData 0
RawSlideData.TCGA_FLAT_NONCONFORMING 288
RawSlideData.MOUSE_GLIOMA 284
Total md5 Processed: 8  

In [ ]:
for cl in archive_db.collection_names():
    if cl.startswith('RawSlideData'):
        print cl, archive_db[cl].count(),
        archive_db[cl].create_index('slide_w_path')  ### Double check everything is indexed as well
        print archive_db[cl].find({'OpenslideFailedOpen': True}).count(),
        print archive_db[cl].find({'OpenSlideCouldOpen': True}).count()
# mydict=dict(b=b, c=c, d=d)
# dview.push(mydict)


In [ ]:
# ## add in code to gneerate/compute md5sum
# import os
# import ipyparallel as ipp

# rc = ipp.Client()
# ar = rc[:].apply_async(os.getpid)
# pid_map = ar.get_dict()

In [ ]:
##Other keys to flag--- noncofirming file name!!
### This will update/add md5sum for records if it doesn't have them
###Start computing the md5sum
for cl in archive_db.collection_names():
    md5_processed = 0
    if cl.startswith('RawSlideData'):
        print cl, archive_db[cl].count()
        slidePtr = list(archive_db[cl].find({ 'md5Sum': {"$exists": False}}))
        print len(slidePtr)

        
            #print r
            
        

In [ ]:
cl = 'RawSlideData.EMORY_BIOBANK'
recToUpdate =  archive_db[cl].find_one({'OpenslideFailedOpen': { "$exists" : False } })

mongoID = recToUpdate['_id']


In [ ]:
for cl in archive_db.collection_names():
    if cl.startswith('RawSlideData'):
        print cl, archive_db[cl].count(),
        ### Now go through and update records
        slidePtr = archive_db[cl].find({'OpenslideFailedOpen': { "$exists" : False }, 'OpenSlideCouldOpen': {"$exists": False} })
        for r in slidePtr:

            ## Fix the OpenSlideCouldOpen
            r['OpenSlideCouldOpen'] = True
            archive_db[cl].replace_one( {'_id':r['_id']},r)



In [ ]:
print archive_db['RawSlideData.WINSHIP_BIOBANK'].find_one({'_id': ObjectId('57b829812f9b2e43e9a5ab40')})

In [ ]:
from bson.objectid import ObjectId

In [ ]:
import sys
import pymongo
import bson.objectid
pymongo.objectid = bson.objectid
sys.modules["pymongo.objectid"] = bson.objectid